<a href="https://colab.research.google.com/github/mananverma29-oss/ai-sales-insights-aws/blob/main/AI_Sales_Insights_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyathena boto3 openai python-dotenv

In [ ]:
# ---- Config ----
RAW_CSV_URL = "https://raw.githubusercontent.com/mananverma29-oss/ai-sales-insights-aws/main/data.csv"
OPENAI_API_KEY = "------CM83vpY3rSdT3BlbkFJtOw0_LuKCSO9baXiZdglfSMRmLPII72I_5f------"

RAW_FILE = "data.csv"
ENHANCED_FILE = "data_enhanced.csv"
AI_SUMMARY_FILE = "ai_summary_output.txt"

print("API key integrated and data.csv loaded")


API key integrated and data.csv loaded


In [ ]:
import requests
r = requests.get(RAW_CSV_URL)
open(RAW_FILE, "wb").write(r.content)
print(f"Saved {RAW_FILE} ({len(r.content)} bytes)")

Saved data.csv (2287806 bytes)


In [ ]:
import pandas as pd, re

df = pd.read_csv(RAW_FILE,encoding='latin1')
print("Columns:", list(df.columns))
print(df.head())


Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  Uni

In [ ]:
df = pd.read_csv(RAW_FILE, encoding='latin1')

def clean_col(c):
    c = str(c).strip()
    c = re.sub(r'[\s/]+', '_', c)
    c = re.sub(r'[^0-9a-zA-Z_]', '', c)
    return c.lower()

df.columns = [clean_col(c) for c in df.columns]

In [ ]:
# Ensure sales column
if 'sales' not in df.columns:
    if 'unitprice' in df.columns and 'quantity' in df.columns:
        df['sales'] = df['unitprice'] * df['quantity']
    else:
        raise SystemExit("No 'sales' column found; please ensure a sales-like column exists.")

In [ ]:
# Fill or derive commonly-missing fields
df['profit']   = df.get('profit', df['sales'] * 0.15)
df['quantity'] = df.get('quantity', 1)
df['discount'] = df.get('discount', 0.0)
df['target']   = df.get('target', df['sales'] * 1.05)

In [ ]:
# Date handling: prefer first column containing 'date'
date_cols = [c for c in df.columns if 'date' in c]
if date_cols:
    df['order_date'] = pd.to_datetime(df[date_cols[0]], errors='coerce').dt.strftime("%Y-%m-%d")
else:
    df['order_date'] = pd.Timestamp.today().strftime("%Y-%m-%d")


In [ ]:
# Reorder and save
cols_order = ['order_date','region','category','sales','profit','quantity','discount','target']
cols_present = [c for c in cols_order if c in df.columns]
df = df[cols_present + [c for c in df.columns if c not in cols_present]]
df.to_csv(ENHANCED_FILE, index=False)
print("Saved", ENHANCED_FILE, "with columns:", list(df.columns))

Saved data_enhanced.csv with columns: ['order_date', 'region', 'category', 'sales', 'profit', 'quantity', 'discount', 'target', 'row_id', 'order_id', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'product_id', 'subcategory', 'product_name']


In [ ]:
output = pd.read_csv(ENHANCED_FILE)
print(output.head())

   order_date region         category     sales    profit  quantity  discount  \
0  2016-11-08  South        Furniture  261.9600   41.9136         2      0.00   
1  2016-11-08  South        Furniture  731.9400  219.5820         3      0.00   
2  2016-06-12   West  Office Supplies   14.6200    6.8714         2      0.00   
3  2015-10-11  South        Furniture  957.5775 -383.0310         5      0.45   
4  2015-10-11  South  Office Supplies   22.3680    2.5164         2      0.20   

        target  row_id        order_id  ... customer_id    customer_name  \
0   275.058000       1  CA-2016-152156  ...    CG-12520      Claire Gute   
1   768.537000       2  CA-2016-152156  ...    CG-12520      Claire Gute   
2    15.351000       3  CA-2016-138688  ...    DV-13045  Darrin Van Huff   
3  1005.456375       4  US-2015-108966  ...    SO-20335   Sean O'Donnell   
4    23.486400       5  US-2015-108966  ...    SO-20335   Sean O'Donnell   

     segment        country             city       state

In [ ]:
print(df['category'])

0             Furniture
1             Furniture
2       Office Supplies
3             Furniture
4       Office Supplies
             ...       
9989          Furniture
9990          Furniture
9991         Technology
9992    Office Supplies
9993    Office Supplies
Name: category, Length: 9994, dtype: object
